In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
buyer_sliders = {
    key: widgets.IntSlider(
        description='{unit} {key}'.format(unit=unit, key=key),
        value=default,
        )
    for unit, default, key in [
        ('$', 90, 'high'),
        ('$/min', 5, 'buyer_time_price'),
        ('$', 10, 'exp_low'),
        ('$/min', 5, 'exp_seller_time_price'),
        ('$', 50, 'buyer_exp_market'),
        ('min', 10, 'buy_time_fungibility'),
        ]
    }

In [3]:
seller_sliders = {
    key: widgets.IntSlider(
        description='{unit} {key}'.format(unit=unit, key=key),
        value=default,
        )
    for unit, default, key in [
        ('$', 10, 'low'),
        ('$/min', 5, 'seller_time_price'),
        ('$', 90, 'exp_high'),
        ('$/min', 5, 'exp_buyer_time_price'),
        ('$', 50, 'seller_exp_market'),
        ('min', 10, 'sell_time_fungibility'),
        ]
    }

In [4]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [5]:
# https://stackoverflow.com/questions/32000934/python-print-a-variables-name-and-value
import sys
DEBUG = False
def debug(expression):
    if DEBUG:
        frame = sys._getframe(1)
        print(expression, ':\t', repr(eval(expression, frame.f_globals, frame.f_locals)))

In [6]:
def buyer_say(text):
    print('\tB: "{}"'.format(text))
    
def seller_say(text):
    print('\t\tS: "{}"'.format(text))

def narrate(text):
    print('\t{}'.format(text))

In [7]:
def sold_at(prices): # list of $, or a single $
    try:
        final_price = mean(prices)
    except TypeError: # a single value
        final_price = prices
    debug('final_price') # $
    narrate('Sold at ${}.'.format(final_price))
    
def calculate(high, buyer_time_price, exp_low, exp_seller_time_price, buyer_exp_market, buy_time_fungibility, low, seller_time_price, exp_high, exp_buyer_time_price, seller_exp_market, sell_time_fungibility):
    TIME_CONSTANT = 1 # min
    
    buyer_null_util = max(0, high - buyer_exp_market - (buyer_time_price * buy_time_fungibility))
    debug('buyer_null_util') # $
    seller_null_util = max(low, seller_exp_market - low - (seller_time_price * sell_time_fungibility))
    debug('seller_null_util') # $
    
    # ladies and gentlemen here's the opening round
    t = 0
    
    seller_bid = [exp_high]
    buyer_bid = [exp_low]
    debug('seller_bid[0]') # $
    debug('buyer_bid[0]') # $
    seller_say('Selling at ${}.'.format(seller_bid[0]))
    buyer_say('Buying at ${}.'.format(buyer_bid[0]))
    
    if buyer_bid[t] >= seller_bid[t]: # CHANGE: allow equal
        sold_at([buyer_bid[t], seller_bid[t]])
        return
    else:
        gap_between_opening_prices = seller_bid[t] - buyer_bid[t]
        debug('gap_between_opening_prices') # $
        
    # second round, let's see if anyone accepts
    buyer_acc_imm_util = high - seller_bid[t]
    seller_acc_imm_util = buyer_bid[t] - low
    debug('buyer_acc_imm_util') # $
    debug('seller_acc_imm_util') # $
    
    # check if the buyer accepts
    if (
            buyer_time_price > gap_between_opening_prices  # BAD COMPARISON ($/min to $)
            and buyer_acc_imm_util > buyer_null_util
            ):
        buyer_say('I accept your offer!')
        buyer_accepts = True
    else:
        buyer_accepts = False
        buyer_bid.append(buyer_bid[t] + (exp_seller_time_price * TIME_CONSTANT))
        debug('buyer_bid[{}]'.format(t+1))
        buyer_say('Buying at ${}.'.format(buyer_bid[t+1]))
        
    
    # check if the seller accepts
    if (
            (seller_time_price * TIME_CONSTANT) > gap_between_opening_prices # CHANGED: corrected by TIME_CONSTANT for units match
            and seller_acc_imm_util > seller_null_util
            ):
        seller_say('I accept your offer!')
        seller_accepts = True
    else:
        seller_accepts = False
        seller_bid.append(seller_bid[t] - (exp_buyer_time_price * TIME_CONSTANT))
        debug('seller_bid[{}]'.format(t+1))
        seller_say('Selling at ${}.'.format(seller_bid[t+1]))
    
    if buyer_accepts and not seller_accepts:
        sold_at(seller_bid[t])
        return
    elif seller_accepts and not buyer_accepts:
        sold_at(buyer_bid[t])
        return
    elif buyer_accepts and seller_accepts:
        sold_at([buyer_bid[t], seller_bid[t]])
        return
    
    # neither accepted, let's do rounds 3+
    #TODO short-circuit based on time preference
    #TODO loop properly
    while buyer_bid[t] < seller_bid[t]:
        t += 1
        buyer_bid.append(buyer_bid[t] + (exp_seller_time_price * TIME_CONSTANT))
        debug('buyer_bid[{}]'.format(t+1))
        buyer_say('Buying at ${}.'.format(buyer_bid[t+1]))
        seller_bid.append(seller_bid[t] - (exp_buyer_time_price * TIME_CONSTANT))
        debug('seller_bid[{}]'.format(t+1))
        seller_say('Selling at ${}.'.format(seller_bid[t+1]))
        
        if t>10:
            narrate('Market closed for the day.')
            return
        
    sold_at([buyer_bid[t], seller_bid[t]])
    return

In [8]:
buyer_box = widgets.VBox([widgets.Label('buyer:')] + list(buyer_sliders.values()))
seller_box = widgets.VBox([widgets.Label('seller:')] + list(seller_sliders.values()))
output_box = widgets.interactive_output(calculate, {k: v for k, v in dict(buyer_sliders, **seller_sliders).items()})

# return a widget with our controls and output
widgets.VBox([widgets.HBox([buyer_box, seller_box]), output_box])